Search Cluster  
----------------------
fecha 25/02/2026

Sirve para identificar a qué cluster pertenece un cajero dado 

Cómo ejecutarlo:
1. Cargar el dataframe histórico con columnas mínimas:
   - cajero
   - fecha
   - retiro
2. Ejecutar la función principal de clasificación.
3. Se generará un dataframe resumen con una fila por cajero.

Qué devuelve:
Un dataframe con:
- cajero
- días observados
- mediana de retiros
- percentil 95 (P95)
- ratio P95 / mediana
- etiqueta de pre-cluster asignada

La etiqueta clasifica el comportamiento estructural del cajero
según tamaño y nivel de dispersión.

En la celda final hay que ingresar el ID del cajero del cual se desea conocer a cuál de los siuientes pre-cluster pertenece,

* NORMAL_ESTABLE
* EVENT_DRIVEN
* NORMAL_CON_PICOS
* GRANDE_Y_ESTABLE

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_parquet('../../Insumos/df_general.parquet')

def clasificar_cajero_por_id(df, cajero_id):
    """
    Devuelve la etiqueta estructural de un cajero específico
    usando la misma lógica data-driven del modelo.
    """
    
    # ==============================
    # 1. Calcular métricas del cajero
    # ==============================
    df_cajero = df[df["cajero"] == cajero_id].copy()
    
    if df_cajero.empty:
        return f"Cajero {cajero_id} no encontrado en dataset."
    
    dias_obs = df_cajero["retiro"].count()
    mediana = df_cajero["retiro"].median()
    p95 = df_cajero["retiro"].quantile(0.95)
    
    if mediana == 0:
        ratio = np.nan
    else:
        ratio = p95 / mediana
    
    # ==============================
    # 2. Recalcular umbrales globales
    # ==============================
    resumen = (
        df
        .groupby("cajero")["retiro"]
        .agg(
            dias_obs="count",
            mediana="median",
            p95=lambda x: x.quantile(0.95),
        )
        .reset_index()
    )
    
    resumen["ratio_p95_mediana"] = resumen["p95"] / resumen["mediana"]
    resumen.loc[resumen["mediana"] == 0, "ratio_p95_mediana"] = np.nan
    
    mask = resumen["mediana"] > 0
    
    umbral_grande = resumen.loc[mask, "mediana"].quantile(0.75)
    umbral_estable = resumen.loc[mask, "ratio_p95_mediana"].quantile(0.50)
    umbral_picos = resumen.loc[mask, "ratio_p95_mediana"].quantile(0.75)
    umbral_evento = resumen.loc[mask, "ratio_p95_mediana"].quantile(0.90)
    
    # ==============================
    # 3. Aplicar la misma lógica de clasificación
    # NOTA- Si la lógica de clasificación cambia, esta sección debe ser también actualizada.
    # ==============================
    if mediana == 0:
        etiqueta = "event_driven"
    elif (mediana >= umbral_grande) and (ratio <= umbral_estable):
        etiqueta = "grande_y_estable"
    elif ratio >= umbral_evento:
        etiqueta = "event_driven"
    elif ratio >= umbral_picos:
        etiqueta = "normal_con_picos"
    else:
        etiqueta = "normal_estable"
    
    # ==============================
    # 4. Resultado estructurado
    # ==============================
    return {
        "cajero": cajero_id,
        "dias_obs": dias_obs,
        "mediana": mediana,
        "p95": p95,
        "ratio_p95_mediana": ratio,
        "etiqueta": etiqueta 
    }

In [7]:
resultado = clasificar_cajero_por_id(df, "JF000001") #F00118J1 NORMAL ESTABLE
print(resultado)

resultado2 = clasificar_cajero_por_id(df, "JF001181") #F00118J1 NORMAL ESTABLE
print(resultado2)

{'cajero': 'JF000001', 'dias_obs': 754, 'mediana': 130950.0, 'p95': 707135.0000000001, 'ratio_p95_mediana': 5.40003818251241, 'etiqueta': 'normal_estable'}
{'cajero': 'JF001181', 'dias_obs': 754, 'mediana': 212800.0, 'p95': 697732.5000000001, 'ratio_p95_mediana': 3.278818139097745, 'etiqueta': 'normal_estable'}
